# MaveDB Score Set Breakdown
This notebook computes the summary statistics used to generate the sankey plot in figure 2a

## Load Relevant Libraries
Run the cell below to load the libraries that are used in this analysis

In [1]:
from os import environ

import pandas as pd

from dcd_mapping.lookup import _normalize_gene

environ["GENE_NORM_DB_URL"] = "postgresql://postgres@localhost:5432/gene_normalizer"

## Load Metadata 
Run the cell below to load metadata information for the examined MaveDB score sets.

In [2]:
mave_dat = pd.read_csv("analysis_files/mave_dat.csv", index_col=[0])
mave_dat = mave_dat.iloc[:-5]
mave_dat.head()

,urn,target_sequence,target_sequence_type,target,assembly_id,uniprot_id,target_type
0,urn:mavedb:00000041-a-1,CTGCGGCTGGAGGTCAAGCTGGGCCAGGGCTGCTTTGGCGAGGTGT...,dna,Src catalytic domain,GCF_000001405.26,P12931,Protein coding
1,urn:mavedb:00000048-a-1,GAGGGGATCAGTATATACACTTCAGATAACTACACCGAGGAAATGG...,dna,CXCR4,GCF_000001405.26,P61073,Protein coding
2,urn:mavedb:00000068-b-1,ATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAGCCCCCTCTGAGTC...,dna,TP53 (P72R),GCF_000001405.26,NaN,Protein coding
3,urn:mavedb:00000045-c-1,ATGGATGTATTCATGAAAGGACTTTCAAAGGCCAAGGAGGGAGTTG...,dna,alpha-synuclein,GCF_000001405.10,P37840,Protein coding
4,urn:mavedb:00000018-a-1,GGTGTCTGTTTGAGGTTGCTAGTGAACACAGTTGTGTCAGAAGCAA...,dna,HBB promoter,GCF_000001405.26,NaN,Regulatory


### Compute the Number of Score Sets with DNA Target Sequences

In [3]:
dna_ts = mave_dat[mave_dat["target_sequence_type"] == "dna"]
f"The number of score sets with DNA target sequences is: {len(dna_ts.index)}"

'The number of score sets with DNA target sequences is: 176'

### Compute the Number of Score Sets with Protein Target Sequences

In [4]:
protein_ts = mave_dat[mave_dat["target_sequence_type"] == "protein"]
f"The number of score sets with protein target sequences is: {len(protein_ts.index)}"

'The number of score sets with protein target sequences is: 33'

### Compute the Number of Protein Coding Score Sets

In [5]:
coding_ss = mave_dat.loc[mave_dat["target_type"] == "Protein coding"]
f"The number of protein coding score sets: {len(coding_ss.index)}"

'The number of protein coding score sets: 168'

### Compute the Number of Regulatory/Other Noncoding Score Sets

In [6]:
noncoding_ss = mave_dat.loc[mave_dat["target_type"] != "Protein coding"]
f"The number of regulatory/other noncoding score sets: {len(noncoding_ss.index)}"

'The number of regulatory/other noncoding score sets: 41'

### Compute the Number of Score Sets with UniProt IDs

In [7]:
f"{len(coding_ss) - len(coding_ss[coding_ss['uniprot_id'].isnull()].index)} score sets have UniProt IDs"

'95 score sets have UniProt IDs'

### Compute the Number of Score Sets with Protein Target Sequences

In [8]:
f"The number of score sets with protein target sequences is: {len(coding_ss[coding_ss['target_sequence_type'] == 'protein'].index)}"

'The number of score sets with protein target sequences is: 33'

### Compute the Number of Regulatory/Other Noncoding Score Sets with DNA Target Sequences

In [9]:
f"The number of regulatory/other noncoding score sets with DNA target sequences is: {len(noncoding_ss[noncoding_ss['target_sequence_type'] == 'dna'].index)}"

'The number of regulatory/other noncoding score sets with DNA target sequences is: 41'

### Compute the Average Length of Protein Target Sequences

In [10]:
protein_filter = coding_ss.loc[coding_ss["target_sequence_type"] == "protein"]
seqlist = list(protein_filter["target_sequence"])
f"The average length of protein target sequences is: {sum(map(len, seqlist)) / len(seqlist)} amino acids"

'The average length of protein target sequences is: 573.060606060606 amino acids'

### Compute the Average Length of Protein Coding DNA Target Sequences

In [11]:
dna_filter = coding_ss.loc[coding_ss["target_sequence_type"] == "dna"]
seqlist = list(dna_filter["target_sequence"])
f"The average length of protein coding DNA target sequences is: {sum(map(len, seqlist)) / len(seqlist)} nucleotides"

'The average length of protein coding DNA target sequences is: 1337.6296296296296 nucleotides'

### Compute the Average Length of Regulatory/Other Noncoding DNA Target Sequences

In [12]:
dna_filter = noncoding_ss.loc[noncoding_ss["target_sequence_type"] == "dna"]
seqlist = list(dna_filter["target_sequence"])
f"The average length of regulatory/other noncoding DNA target sequences is: {sum(map(len, seqlist)) / len(seqlist)} nucleotides"

'The average length of regulatory/other noncoding DNA target sequences is: 353.2682926829268 nucleotides'

### Compute the Number of Protein Coding Score Sets with Gene Symbol/Aliases

In [13]:
td = list(coding_ss["target"])
td = [x for x in td if " " not in x]
f"The number of protein coding score sets with gene symbols/aliases is: {len(td)}"

'The number of protein coding score sets with gene symbols/aliases is: 125'

### Compute the Number of Protein Coding Score Sets with Descriptive Targets

In [14]:
td = list(coding_ss["target"])
td = [x for x in td if " " in x]
f"The number of protein coding score sets with descriptive targets is: {len(td)}"

'The number of protein coding score sets with descriptive targets is: 43'

### Compute the Number of Regulator/Other Noncoding Score Sets with Descriptive Targets

In [15]:
td = list(noncoding_ss["target"])
td = [x for x in td if " " in x]
f"The number of regulatory/other noncoding score sets with descriptive targets is: {len(td)}"

'The number of regulatory/other noncoding score sets with descriptive targets is: 41'

### Compute the Number of Unique Genes that are Mapped Across Score Sets

In [16]:
targets = list(mave_dat["target"])
unique_genes = []
for gene in targets:
    comp = gene.split()
    if "Minigene" in comp:
        unique_genes.append("WT1")
    elif "hYAP65" in comp:
        unique_genes.append("YAP1")
    else:
        for ele in comp:
            is_normalized = False
            if _normalize_gene(ele):
                unique_genes.append(_normalize_gene(ele).label)
                is_normalized = True
                break
        if not is_normalized:
            unique_genes.append(comp[0])
f"The number of unique gene symbols across examined score set targets is: {len(set(unique_genes))}"

'The number of unique gene symbols across examined score set targets is: 71'